In [55]:
import scipy
import openpyxl
import pandas as pd
import numpy as np

In [ ]:
path = "Data_2024.xlsx"
wb = openpyxl.load_workbook(path)
sheet = wb["Data_věrohodnost"]


data = []

uncensored_data = []
censored_data = []

for row in sheet.iter_rows(min_row=2, max_col=2, values_only=True):
    data.append(row[1])
    
    if(row[0] == 0):
        uncensored_data.append(row[1])
    else:
        censored_data.append(row[1])
    
uncensored_data = np.array(uncensored_data)
censored_data = np.array(censored_data)
print(uncensored_data)



In [57]:
def log_likelihood(params, uncensored_data, censored_data):
    k, lambd = params
    
    ll_uncensored = np.sum(np.log(k/lambd) + np.log((uncensored_data/lambd)**(k-1)) - (uncensored_data/lambd)**k)
    
    
    ll_censored = np.sum(-(censored_data / lambd)**k)

    return -(ll_uncensored + ll_censored)


In [ ]:
import scipy.optimize


initial_guess = [1.5, np.mean(data)]

#result = scipy.optimize.minimize(log_likelihood, initial_guess, args=(uncensored_data, censored_data), method='L-BFGS-B', bounds=[(0.01, None), (0.01, None)])

result = scipy.optimize.minimize(fun=log_likelihood, x0=initial_guess, args=(uncensored_data, censored_data), method='BFGS')

# Výsledky
k_opt, lambd_opt = result.x
print(f"est. params: k = {k_opt:.4f}, lambda = {lambd_opt:.4f}")